# CERVVAL

<b>Project subSEE4D</b><br>
Physical-based underwater image simulation<br>
    
<b>Phase 1: Simplified simulator (optimized)</b><br>
    
Author: Ricardo MENDOZA<br>
June 2020<br>

<br>    
<b>Description:</b>
<p>
Comprises a simplified parametrization and modeling of color attenuation and turbidity degradation*; based on a wavelength-dependent exponential color attenuation model (Eq. 1), and a simplified probabilistic model for turbidity-associated image degradation (Lambertian-based diffusion/blurring) (Fig. 2, Eq. 2). No specific environmental scenarios are considered (for instance, surface ice or snow).

*This model variant also considers parameters for turbidity-induced noise (texture) and particle color.
</p>

</p>
3 main parameters are considered:
per-channel water attenuation coefficients (RGB);
a global, constant, ambient light source (described by its RGB values)
per channel (RGB) scattering probability densities, governing photon scattering. Both attenuation and turbidity properties are treated as constant throughout the water body. No modeling of the camera sensors and optics is considered. The simulator produces a degraded (RGB) image (only) from the RGB+D input pair and parameters.
</p>
<br>
<br>
(1)
$$I_c(x)=t_c(x)J_c(x) + (1-t_c(x))\cdot A_c$$
<br>

![](res/Fig1_v3.png)
Fig. 1. Simple scattering model using a distance-parameterized sensor-spatial distribution (hazing - blurring, and photon loss). A normal distribution (Eq. 2) seems to be a natural approximation for an isotropic, lambertian, water-body constant density of homogeneous particles.
<br>
<br>
(2)
$$ P_{s}^d(s') = 
\frac{1}{ 2\pi(\gamma_c d_s)}
exp{\left( -\frac{(s_1'-s_1)^2 + (s_2'-s_2)^2}{2(\gamma_c d_s)^2} \right)}$$
<br>
<br>
The agregation of scattering distributions from all input image pixels gives the expected scattered signal recovered for a particular output pixel $I^{out}(s')$ (Eq. 3).
<br>
<br>
(3)
$$I_c^{out}(s) = \int_{s' \in S} e^{-\alpha_c d_{s'}} I_c(s') P_{s'}^{d_s}(s) $$
<br>
where: <br>
- $S$ is the set of input image pixels<br> 
- $\gamma_c$ modulates the variance of the scattering relationship with distance for channel $c$ (a proxy for turbidity density);<br>
- $\alpha_c$ models the loss due to unrecoverable scattered photons for channel $c$ (a proxy for reflective characteristics of turbid medium)<br>
<br>
<br>
The total signal arriving at output pixel $s'$ which accounts for color attenuation + scattering due turbidity $I^{tot}_c(s')$ is modeled by Eq. 4.
<br>
<br>
(4)
$$I^{tot}_c(s)= ( I^{out}(s) + (1-e^{-\alpha_c d_{s}} )I(s) ) e^{-\beta_c d_s} + 
(1-e^{-\beta_c d_s}) \cdot A_c$$
<br>
Note: Eq. 4 does not take into account scattering effects on the ambient (veiling) light $A_c$


In [1]:
from matplotlib import pyplot as plt
import numpy as np
import cv2

import torch
import torch.nn.functional as F
from torch.autograd import Variable

import time

### from matplotlib import pyplot as plt
import numpy as np
import cv2

import torch
import torch.nn.functional as F
from torch.autograd import Variable

import time



In [2]:
#Parameters
#pExp = 5.0/(1*5*5*5*5)
gamma = [0.03, 0.009, 0.043]
alpha = [0.012, 0.012, 0.012]
beta = [0.020, 0.005, 0.010]
A_light = [00.0, 120.0, 30.0]
turbu_p = [0.15, 1.5] # noise amount, est. dev.
turbu_c = [34.0, 200.0, 201.0]
u = 0.98 #weighting between scattering + attenuation image vs particle noise image 
s = 5 # multiplier for particle noise image
depht_add = 80 #addition to values of depht map (a minimum distance for map)
depht_levels = 16
kern_size = 10


#### Create separate images corresponding to n depht levels and corresponding kernel (for scattering dist.) 

In [3]:
#Compute scatering diffusion for all pixels (Eq. 2, 3) using depht levels
def scatterPsdOp(rgb, depht, gamma, alpha, win_size, d_levels):

    dim = rgb.shape
    psd = np.zeros(dim)

    d_min = torch.min(depht)
    d_max = torch.max(depht)
    
    img_sum = torch.zeros(dim[0], dim[1], 3)
    
    for n in range(d_levels):
        img_l = torch.zeros(dim[0], dim[1])
        ker = torch.zeros(2*win_size + 1, 2*win_size + 1)
        
        di_min = ((d_max-d_min)/d_levels)*n + d_min
        di_max = ((d_max-d_min)/d_levels)*(n+1) + d_min
        di_m = (di_min + di_max)/2
        
        for c in range(3):
            
            # compute kernel
            for i in range(2*win_size + 1):
                for j in range(2*win_size + 1):
                    pos = [i - win_size, j - win_size] 
                    v0 = -( np.power(pos[0], 2) + np.power(pos[1], 2) )
                    v1 = np.exp( v0 / ( 2*np.power(gamma[c]*di_m, 2) ) )                
                    v2 = 1 / ( (2*np.pi) * np.power(gamma[c]*di_m, 1) ) 
                    v3 = v1*v2
                    ker[i, j] = v3

            ker = ker/torch.sum(ker)        

            #extract pixels in depht range
            img_t = rgb[:, :, c]          
            dht = (depht >= di_min ).data.numpy().astype(np.float32)
            dht = np.multiply(dht, (depht < di_max ).data.numpy().astype(np.float32))
            img_l = torch.from_numpy(np.multiply(dht, img_t.data.numpy()))
            
            #Convolve            
            va = Variable(ker.view(1,1,2*win_size + 1, 2*win_size + 1))
            vb = Variable(img_l.view(1,1, dim[0], dim[1]))
            img_c = F.conv2d(vb, va, padding=win_size)
            img_c = img_c[0, 0, :, :]
            
            img_sum[:, :, c] += img_c
            img_sum[img_sum > 255] = 255 #for saturated values
        
    return img_sum

    
    

#### Compute scattered signal map

In [4]:
#Compute degradation caused by colored particle turbidity
def turbidDeg(rgb, depht, turbu_p, turbu_c):
    dim = rgb.shape
    s_vs_p = 0.5
    out = np.zeros(rgb.shape)
    
    # Salt mode
    num_salt = np.ceil(turbu_p[0] * rgb.size * s_vs_p * 3)
    coords = [np.random.randint(0, i - 1, int(num_salt))
        for i in rgb.shape]
    out[coords] = 1.0
    
    out[:, :, 0] = out[:, :, 0]*turbu_c[0] 
    out[:, :, 1] = out[:, :, 1]*turbu_c[1] 
    out[:, :, 2] = out[:, :, 2]*turbu_c[2]
    
    #pass a gaussian filter (blurring) to noisy points
    #kernel = np.ones((5,5),np.float32)/25
    #out = cv2.filter2D(out,-1,kernel)
    out = cv2.GaussianBlur(out,(9,9),turbu_p[1])
    
    return out


#Compute total arriving intensity for a pixel (Eq. 4)
def outTotal(rgb, depht, I_out, gamma, alpha, beta, A_light):
      
    dim = rgb.shape    
    I_total = np.zeros(rgb.shape)
    #I_out = np.zeros(rgb.shape)
    
    for c in range(3):
        for i in range(0, dim[0], 1):
            for j in range(0, dim[1], 1):
                s = [i, j]
                v1 = np.exp(-alpha[c]*depht[s[0]][s[1]])*rgb[s[0]][s[1]][c]
                v2 = np.exp(-beta[c]*depht[s[0]][s[1]])
                I_total[s[0]][s[1]][c] = (I_out[s[0]][s[1]][c] + v1)*v2 + (1-v2)*A_light[c]

    return I_total

In [5]:
def processImg(imgD_Norm, imgRGB):

    imgD_Norm += depht_add #add a minimum to depth map

    imgD_Norm_T = torch.from_numpy(imgD_Norm)
    imgRGB_T = torch.from_numpy(imgRGB)

    #Compute scattering part
    I_out = scatterPsdOp(imgRGB_T, imgD_Norm_T, gamma, alpha, kern_size, depht_levels)
    I_out = I_out.data.numpy()

    #Compute total model (scattering & loss + attenuation + ambient) 
    I_total = outTotal(imgRGB.astype(np.float32), imgD_Norm.astype(np.float32), I_out, gamma, alpha, beta, A_light)

    # particle turbidity effect 
    dim = I_total.shape
    out = np.zeros(dim)
    out2 = turbidDeg(imgRGB.astype(np.float32), imgD_Norm.astype(np.float32), turbu_p, turbu_c)
    out =  I_total*u + out2*s*(1-u)

    #Adjust and save result
    import math
    for c in range(3):
        for i in range(dim[0]):
            for j in range(dim[1]):
                if math.isnan(out[i][j][c]):
                    out[i][j][c] = 255
                else:
                    out[i][j][c] = out[i][j][c]

                if out[i][j][c] > 255:
                    out[i][j][c] = 255
                else:
                    out[i][j][c] = out[i][j][c]

    out = out.astype(np.uint8)
    
    return out
    

In [6]:
def getRndPar():
    r = np.random.randint(1, 20, size=1)
    rb = np.random.randint(1, 300, size=1)
    val = (1.0/2500000.0) * 16 #  r[0]*r[0]*r[0]*r[0]
    val = val*(1.0 + (rb[0]/1000.0))
    return val


### To process all images

In [ ]:
import glob

tt = time.process_time()

outpath = '/home/tmondal/Documents/Dataset/Dehazing/NYU_Depth_V2'

list_f = glob.glob('../../../../NYU_Depth_V2/Extract/*_img.png')
list_f = sorted(list_f)

count = 1
# for epch in range(50):
#     for f_rgb in list_f:

t_img = time.process_time()

# f_dep = f_rgb.replace("_img", "_deNorm")


beta = [getRndPar(), getRndPar(), getRndPar()] 


f_rgb = '/home/tmondal/Documents/Dataset/Dehazing/NYU_Depth_V2/test_rgb_1.png'
f_dep = '/home/tmondal/Documents/Dataset/Dehazing/NYU_Depth_V2/test_depth_1.png'
#Open images
imgRGB = cv2.imread(f_rgb,cv2.IMREAD_COLOR)
imgRGB = cv2.cvtColor(imgRGB, cv2.COLOR_BGR2RGB)

imgD_Norm = cv2.imread(f_dep,cv2.IMREAD_ANYDEPTH)
imgD_Norm = imgD_Norm + 1
imgD_Norm = (imgD_Norm/np.min([np.max(imgD_Norm), 255]))*255  # values are > 0 & <= 255

print(str(count) + '/' + str(len(list_f)) + ' - ' + f_rgb) 
#plt.imshow(imgRGB)
#plt.title('RGB')
#plt.show()
#plt.imshow(imgD_Norm)
#plt.title('Depth')
#plt.show()

#Process
out = processImg(imgD_Norm, imgRGB)

#Show
#print(np.max(out.astype(np.uint8)))
#plt.imshow(out.astype(np.uint8))
#plt.title('out')
#plt.show()

#Save
out = cv2.cvtColor(out, cv2.COLOR_BGR2RGB)

# fname = f_rgb.replace("../../../../NYU_Depth_V2/Extract/", "")
# fname = fname.replace("_img.png", "")


# fname = outpath + fname 
#    fname+= '_g' +str(gamma)
#    fname+= '-a' + str(alpha)
#    fname+= '-b' + str(beta)
#    fname+= '-A' + str(A_light)
#    fname+= '-u' + str(u)
#    fname+= '-add' + str(depht_add)
#    fname+= '-n' + str(depht_levels)
#    fname+= '-r' + str(kern_size)
#    fname+= '-pExp' + str(pExp)
fname = '/home/tmondal/Documents/Dataset/Dehazing/NYU_Depth_V2/test_generated_1.png'
print(fname)
cv2.imwrite(fname, out.astype(np.uint8))

#Create parameter file
'''
file1 = open(fname.replace(".png", ".txt"),"a")
file1.write( str(beta[0]) + "\t" + str(beta[1]) + "\t" + str(beta[2]) + '\n')
file1.write( str(alpha[0]) + "\t" + str(alpha[1]) + "\t" + str(alpha[2]) + '\n')
file1.write( str(gamma[0]) + "\t" + str(gamma[1]) + "\t" + str(gamma[2]) + '\n')
file1.write( str(A_light[0]) + "\t" + str(A_light[1]) + "\t" + str(A_light[2]) + '\n')
file1.write( str(turbu_p[0]) + "\t" + str(turbu_p[1]) + '\n')
file1.write( str(turbu_c[0]) + "\t" + str(turbu_c[1]) + "\t" + str(turbu_c[2]) + '\n')
file1.write( str(u) + "\t" + str(s) + '\n')
file1.write( str(depht_add) + '\n')
file1.write( str(depht_levels) + '\n')
file1.write( str(kern_size) + '\n')
file1.flush()
file1.close()
'''

# count+=1
# print("Image Time: " + str(time.process_time() - t_img) )

    
# print("Total Time: " + str(time.process_time() - tt) + " --> " +  str(len(list_f)) + " images") 



1/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10001_img.png


/home/ram/Work/venv/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  # This is added back by InteractiveShellApp.init_path()


../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10001_TuOp_v11_000.png
Image Time: 26.419525626000002
2/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10002_img.png
../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10002_TuOp_v11_000.png
Image Time: 26.35112334
3/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10003_img.png
../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10003_TuOp_v11_000.png
Image Time: 26.556335282
4/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10004_img.png
../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10004_TuOp_v11_000.png
Image Time: 25.574127462000007
5/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10005_img.png
../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10005_TuOp_v11_000.png
Image Time: 25.830353918
6/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10006_img.png
../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10006_TuOp_v11_000.png
Image Time: 26.836372382000008
7/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_1

../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10049_TuOp_v11_000.png
Image Time: 26.572391411999888
50/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10050_img.png
../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10050_TuOp_v11_000.png
Image Time: 26.207797164999874
51/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10051_img.png
../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10051_TuOp_v11_000.png
Image Time: 26.21774753999989
52/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10052_img.png
../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10052_TuOp_v11_000.png
Image Time: 26.515179489000047
53/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10053_img.png
../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10053_TuOp_v11_000.png
Image Time: 25.91386011099985
54/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10054_img.png
../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10054_TuOp_v11_000.png
Image Time: 26.724715553000124
55/1449 - ../../../../NYU_D

../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10097_TuOp_v11_000.png
Image Time: 26.071894631000305
98/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10098_img.png
../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10098_TuOp_v11_000.png
Image Time: 25.740841896000347
99/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10099_img.png
../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10099_TuOp_v11_000.png
Image Time: 26.11000621500034
100/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10100_img.png
../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10100_TuOp_v11_000.png
Image Time: 26.01819311899999
101/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10101_img.png
../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10101_TuOp_v11_000.png
Image Time: 25.527214163999815
102/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10102_img.png
../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10102_TuOp_v11_000.png
Image Time: 25.608095846000197
103/1449 - ../../../../N

...

../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10241_TuOp_v11_000.png
Image Time: 28.17778274400007
242/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10242_img.png
../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10242_TuOp_v11_000.png
Image Time: 28.416206337999938
243/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10243_img.png
../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10243_TuOp_v11_000.png
Image Time: 27.818171508999512
244/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10244_img.png
../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10244_TuOp_v11_000.png
Image Time: 28.438794040000175
245/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10245_img.png
../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10245_TuOp_v11_000.png
Image Time: 28.475771229999737
246/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10246_img.png
../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10246_TuOp_v11_000.png
Image Time: 27.76192347199958
247/1449 - ../../../..

../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10289_TuOp_v11_000.png
Image Time: 28.131501801000013
290/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10290_img.png
../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10290_TuOp_v11_000.png
Image Time: 28.360212780999973
291/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10291_img.png
../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10291_TuOp_v11_000.png
Image Time: 28.228086252999674
292/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10292_img.png
../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10292_TuOp_v11_000.png
Image Time: 28.833801804999894
293/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10293_img.png
../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10293_TuOp_v11_000.png
Image Time: 28.072715217999757
294/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10294_img.png
../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10294_TuOp_v11_000.png
Image Time: 28.59611330699954
295/1449 - ../../../.

../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10337_TuOp_v11_000.png
Image Time: 28.22208154100008
338/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10338_img.png
../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10338_TuOp_v11_000.png
Image Time: 29.293736559000536
339/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10339_img.png
../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10339_TuOp_v11_000.png
Image Time: 28.57551325699933
340/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10340_img.png
../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10340_TuOp_v11_000.png
Image Time: 28.195099841999763
341/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10341_img.png
../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10341_TuOp_v11_000.png
Image Time: 27.696260847000303
342/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10342_img.png
../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10342_TuOp_v11_000.png
Image Time: 28.488144777000343
343/1449 - ../../../..

../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10385_TuOp_v11_000.png
Image Time: 28.32417792400156
386/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10386_img.png
../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10386_TuOp_v11_000.png
Image Time: 27.89513156400062
387/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10387_img.png
../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10387_TuOp_v11_000.png
Image Time: 28.13917540700095
388/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10388_img.png
../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10388_TuOp_v11_000.png
Image Time: 28.75416889799999
389/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10389_img.png
../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10389_TuOp_v11_000.png
Image Time: 28.398550584000986
390/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10390_img.png
../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10390_TuOp_v11_000.png
Image Time: 27.954006467
391/1449 - ../../../../NYU_Dep

../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10433_TuOp_v11_000.png
Image Time: 28.42207544899975
434/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10434_img.png
../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10434_TuOp_v11_000.png
Image Time: 28.532863570999325
435/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10435_img.png
../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10435_TuOp_v11_000.png
Image Time: 28.246513770000092
436/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10436_img.png
../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10436_TuOp_v11_000.png
Image Time: 28.168319403999703
437/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10437_img.png
../../../../NYU_Depth_V2/exp_cyclegan_v3/NYUv2_r2/NYUv2_10437_TuOp_v11_000.png
Image Time: 28.85461234899958
438/1449 - ../../../../NYU_Depth_V2/Extract/NYUv2_10438_img.png
